#  R(データミックスのインテグレーションステップ課題　エンジンの予知保全　全データ一括での生存時間分析による予測.R)において生存時間分析で予測を行ったデータを読み込み、混合行列を作成する

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.metrics import confusion_matrix, accuracy_score,precision_score,recall_score, f1_score,classification_report
from random import randint, shuffle
import os
import seaborn as sns




# 閾値0.5

# Rで作成した予測付きのtestデータを読み込む

In [2]:
test_data_R_time_step_0_5 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_time_step_0.5.csv')

In [3]:
test_data_R_time_step_0_5.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_step_flag
0,1,0,1,42.0034,0.8400,100,445.00,549.36,1342.05,1124.56,...,0.02,329,2212,100.0,10.69,6.3956,2,-0.541003,0.367954,0
1,2,0,2,42.0017,0.8400,100,445.00,548.83,1351.93,1116.28,...,0.02,331,2212,100.0,10.55,6.3775,3,-0.534184,0.369542,0
2,3,0,3,0.0028,0.0019,100,518.67,642.35,1583.74,1400.44,...,0.03,392,2388,100.0,38.85,23.3483,4,-0.359101,0.411177,0
3,4,0,4,42.0047,0.8400,100,445.00,549.69,1354.36,1125.55,...,0.02,330,2212,100.0,10.56,6.4871,5,-0.314766,0.421952,0
4,5,0,5,10.0058,0.2506,100,489.05,604.72,1496.65,1310.52,...,0.03,370,2319,100.0,28.78,17.1987,6,-0.321387,0.420338,0


In [4]:
#結果表の作成 
test_data_R_time_step_0_5_pred = test_data_R_time_step_0_5.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_time_step_0_5_pred = test_data_R_time_step_0_5_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_step_flag']]



In [5]:
test_data_R_time_step_0_5_pred.head()

,engine_no,pred_test2_time_step_flag
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1


In [23]:
#列名変換

In [24]:
#csvで結果の保存
# result_test_data_all_pred[['engine_no', 'result']].to_csv('Downloads/インテグ課題/NASA/prehackathonsup/result_test_data_all_pred.csv', index=False)

# test_dataのRULを読み込んで処理

In [6]:
#testデータのRULを読み込む
test_data_RUL = pd.read_csv('Downloads/インテグ課題/NASA/prehackathonsup/RUL_test.csv')

In [7]:
test_data_RUL.head()

,Unnamed: 0,RUL,engine_no
0,0,22,0
1,1,12,1
2,2,108,2
3,3,88,3
4,4,10,4


In [8]:
#列の順番を入れ替えと削除
test_data_RUL.drop('Unnamed: 0', axis=1)
test_data_RUL = test_data_RUL.loc[:,['engine_no','RUL']]
test_data_RUL.head()

,engine_no,RUL
0,0,22
1,1,12
2,2,108
3,3,88
4,4,10


In [9]:
# RUL<100の時、つまり予測した残りの寿命サイクルが100より小さければ1 それ以外で0とする
test_data_RUL['result'] = test_data_RUL['RUL'].map(lambda x: 0 if x > 100 else 1)

In [10]:
test_data_RUL.head()

,engine_no,RUL,result
0,0,22,1
1,1,12,1
2,2,108,0
3,3,88,1
4,4,10,1


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [11]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL['result'],test_data_R_time_step_0_5_pred['pred_test2_time_step_flag'],labels=[1, 0] )

In [12]:
cm

array([[348, 102],
       [ 80, 177]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [13]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL['result'],test_data_R_time_step_0_5_pred['pred_test2_time_step_flag'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.81      0.77      0.79       450
           0       0.63      0.69      0.66       257

    accuracy                           0.74       707
   macro avg       0.72      0.73      0.73       707
weighted avg       0.75      0.74      0.74       707



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方

# 閾値0.4

# Rで作成した予測付きのtestデータを読み込む

In [14]:
test_data_R_time_step_0_4 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_time_step_0.4.csv')

In [15]:
test_data_R_time_step_0_4.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_step_flag
0,1,0,1,42.0034,0.8400,100,445.00,549.36,1342.05,1124.56,...,0.02,329,2212,100.0,10.69,6.3956,2,-0.541003,0.367954,0
1,2,0,2,42.0017,0.8400,100,445.00,548.83,1351.93,1116.28,...,0.02,331,2212,100.0,10.55,6.3775,3,-0.534184,0.369542,0
2,3,0,3,0.0028,0.0019,100,518.67,642.35,1583.74,1400.44,...,0.03,392,2388,100.0,38.85,23.3483,4,-0.359101,0.411177,1
3,4,0,4,42.0047,0.8400,100,445.00,549.69,1354.36,1125.55,...,0.02,330,2212,100.0,10.56,6.4871,5,-0.314766,0.421952,1
4,5,0,5,10.0058,0.2506,100,489.05,604.72,1496.65,1310.52,...,0.03,370,2319,100.0,28.78,17.1987,6,-0.321387,0.420338,1


In [16]:
#結果表の作成 
test_data_R_time_step_0_4_pred = test_data_R_time_step_0_4.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_time_step_0_4_pred = test_data_R_time_step_0_4_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_step_flag']]



In [17]:
test_data_R_time_step_0_4_pred.head()

,engine_no,pred_test2_time_step_flag
0,0,1
1,1,1
2,2,0
3,3,1
4,4,1


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [18]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL['result'],test_data_R_time_step_0_4_pred['pred_test2_time_step_flag'],labels=[1, 0] )

In [19]:
cm

array([[419,  31],
       [162,  95]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [20]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL['result'],test_data_R_time_step_0_4_pred['pred_test2_time_step_flag'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.72      0.93      0.81       450
           0       0.75      0.37      0.50       257

    accuracy                           0.73       707
   macro avg       0.74      0.65      0.65       707
weighted avg       0.73      0.73      0.70       707



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方

# 閾値0.6

# Rで作成した予測付きのtestデータを読み込む

In [21]:
test_data_R_time_step_0_6 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_time_step_0.6.csv')

In [22]:
test_data_R_time_step_0_6.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_step_flag
0,1,0,1,42.0034,0.8400,100,445.00,549.36,1342.05,1124.56,...,0.02,329,2212,100.0,10.69,6.3956,2,-0.541003,0.367954,0
1,2,0,2,42.0017,0.8400,100,445.00,548.83,1351.93,1116.28,...,0.02,331,2212,100.0,10.55,6.3775,3,-0.534184,0.369542,0
2,3,0,3,0.0028,0.0019,100,518.67,642.35,1583.74,1400.44,...,0.03,392,2388,100.0,38.85,23.3483,4,-0.359101,0.411177,0
3,4,0,4,42.0047,0.8400,100,445.00,549.69,1354.36,1125.55,...,0.02,330,2212,100.0,10.56,6.4871,5,-0.314766,0.421952,0
4,5,0,5,10.0058,0.2506,100,489.05,604.72,1496.65,1310.52,...,0.03,370,2319,100.0,28.78,17.1987,6,-0.321387,0.420338,0


In [23]:
#結果表の作成 
test_data_R_time_step_0_6_pred = test_data_R_time_step_0_6.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_time_step_0_6_pred = test_data_R_time_step_0_6_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_step_flag']]



In [24]:
test_data_R_time_step_0_6_pred.head()

,engine_no,pred_test2_time_step_flag
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [25]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL['result'],test_data_R_time_step_0_6_pred['pred_test2_time_step_flag'],labels=[1, 0] )

In [26]:
cm

array([[243, 207],
       [  9, 248]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [29]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL['result'],test_data_R_time_step_0_6_pred['pred_test2_time_step_flag'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.96      0.54      0.69       450
           0       0.55      0.96      0.70       257

    accuracy                           0.69       707
   macro avg       0.75      0.75      0.69       707
weighted avg       0.81      0.69      0.69       707



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方